Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
from shutil import copyfile
from shutil import move

In [3]:

path = "data/"
#path = "data/sample/"

In [4]:
batch_size=64

# Create Validation Set
Determine number of files


In [6]:
%cd data/train

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/train


In [7]:
g = glob('c?/*.jpg')

In [8]:
len(g)

22424

In [9]:
shuf = np.random.permutation(g)

In [15]:
validation_size = int(len(g)//6.5)

In [16]:
validation_size

3449

In [12]:
%mkdir ../valid

In [13]:
for d in glob('c?'):
    os.mkdir('../valid/'+d)

In [17]:
for i in range(validation_size): move(shuf[i], '../valid/' + shuf[i])

In [19]:
%cd ..

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data


## Setup batches

In [5]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 18975 images belonging to 10 classes.
Found 3449 images belonging to 10 classes.


In [6]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 79726 images belonging to 1 classes.


In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18975 images belonging to 10 classes.
Found 3449 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [8]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 22424 images belonging to 10 classes.


KeyboardInterrupt: 

## Imagenet Conv Features

Use pre-trained VGG weights to help

So we can pre-compute the output of the last convolutional layer (see lesson 3)  

However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.

In [8]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [6]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [9]:
conv_model = Sequential(conv_layers)

In [8]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18975 images belonging to 10 classes.
Found 3449 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [11]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)

In [12]:
save_array(path+'results/conv_feat.dat', conv_feat)

In [13]:
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [14]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [15]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [16]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [10]:
conv_feat = load_array(path+'results/conv_feat.dat')

In [17]:
conv_feat.shape

(18975, 512, 14, 14)

In [11]:
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [18]:
conv_val_feat.shape

(3449, 512, 14, 14)

## Batchnorm dense layers on pretrained conv layers

We've pre-computed the output of the last convolutional layer, need to create a network that takes that as input and predict our 10 classes.

We can try a simple version of VGG's dense layers


### TO DO HERE

remember to save the weights!!!!!!!!

In [19]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10,activation='softmax')
    ]

In [29]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ]

In [30]:
p=0.8

In [33]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_8 (MaxPooling2D)    (None, 512, 7, 7)     0           maxpooling2d_input_3[0][0]       
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 25088)         0           maxpooling2d_8[0][0]             
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 200)           5017800     flatten_4[0][0]                  
____________________________________________________________________________________________________
batchnormalization_4 (BatchNormal(None, 200)           400         dense_9[0][0]                    
___________________________________________________________________________________________

In [35]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1,
            validation_data=(conv_val_feat, val_labels))

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock
Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=broadwell -mmmx -mno-3dnow -msse -msse2 -msse3 -mssse3 -mno-sse4a -mcx16 -msahf -mmovbe -maes -mno-sha -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -mbmi2 -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mrtm -mhle -mrdrnd -mf16c -mfsgsbase -mno-rdseed -mno-prfchw -mno-adx -mfxsr -mxsave -mxsaveopt -mno-avx512f -mno-avx512er -mno-avx512cd -mno-avx512pf -mno-prefetchwt1 -mno-clflushopt -mno-xsavec -mno-xsaves -mno-avx512dq -mno-avx512bw -mno-avx512vl -mno-avx512ifma -mno-avx512vbmi -mno-clwb -mno-pcommit -mno-mwaitx --param l1-cache-size=32 --param l1-cache-line-size=64 --param l2-cache-size=46080 -mtune=broadwell -DNPY_NO_DEPRECATED_API=NP

OSError: [Errno 12] Cannot allocate memory

In [23]:
bn_model.optimizer.lr=0.01

In [27]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=20,
            validation_data=(conv_val_feat, val_labels))

Train on 18975 samples, validate on 3449 samples
Epoch 1/20
18975/18975 [==============================] - 8s - loss: 2.2270 - acc: 0.1814 - val_loss: 2.3189 - val_acc: 0.1247
Epoch 2/20
18975/18975 [==============================] - 8s - loss: 2.2143 - acc: 0.1822 - val_loss: 2.3393 - val_acc: 0.1206
Epoch 3/20
18975/18975 [==============================] - 8s - loss: 2.2022 - acc: 0.1921 - val_loss: 2.3391 - val_acc: 0.1212
Epoch 4/20
18975/18975 [==============================] - 8s - loss: 2.1914 - acc: 0.1998 - val_loss: 2.3516 - val_acc: 0.1131
Epoch 5/20
18975/18975 [==============================] - 8s - loss: 2.1848 - acc: 0.2083 - val_loss: 2.3511 - val_acc: 0.1038
Epoch 6/20
18975/18975 [==============================] - 8s - loss: 2.1687 - acc: 0.2133 - val_loss: 2.3516 - val_acc: 0.1081
Epoch 7/20
18975/18975 [==============================] - 8s - loss: 2.1538 - acc: 0.2220 - val_loss: 2.3730 - val_acc: 0.1055
Epoch 8/20
18975/18975 [==============================] - 8s -

In [28]:
bn_model.save_weights(path+'models/conv8.h5')

## Pre-computed data augmentation + dropout

In [19]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 22424 images belonging to 10 classes.


We use those to create a dataset of convolutional features
3x bigger than the training set.

In [13]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*3)

In [20]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample)

daconv_feat2.dat is 5x the training set.
save_array(path+'results/daconv_feat2.dat', da_conv_feat)

In [14]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [21]:
save_array(path+'results/da_conv_feat2a.dat', da_conv_feat)
## with 1 batch

In [12]:
da_conv_feat = load_array(path+'results/da_conv_feat2a.dat')
## with 1 batch

In [7]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

combine the real training data as well in its non-augmented form

In [13]:
da_conv_feat.shape

(22424, 512, 14, 14)

In [14]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [15]:
save_array(path+'results/da_conv_feat_concate2x.dat', da_conv_feat)

Since we have 6x bigger of dataset.  We need to copy the labels 6 times as well.

In [16]:
trn_labels.shape

(22424, 10)

In [17]:
da_trn_labels = np.concatenate([trn_labels]*2)

In [18]:
da_trn_labels.shape

(44848, 10)

Since we pre-computed the output of the last convolutional layers from VGG 
we need to create a network that takes that as the input ad predicts our 10 classes
let's try using a simplied version of VGG's dense layer.

In [5]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [6]:
p=0.8

In [12]:
bn_model=Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/4
44848/44848 [==============================] - 21s - loss: 1.8490 - acc: 0.3342 - val_loss: 2.2465 - val_acc: 0.2417
Epoch 2/4
44848/44848 [==============================] - 20s - loss: 1.7789 - acc: 0.3589 - val_loss: 2.2749 - val_acc: 0.1262
Epoch 3/4
44848/44848 [==============================] - 20s - loss: 1.7216 - acc: 0.3782 - val_loss: 2.2834 - val_acc: 0.1287
Epoch 4/4
44848/44848 [==============================] - 21s - loss: 1.6783 - acc: 0.3954 - val_loss: 2.2947 - val_acc: 0.1162


In [28]:
bn_model.optimizer.lr=0.01

In [29]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/4
44848/44848 [==============================] - 21s - loss: 1.6534 - acc: 0.4054 - val_loss: 2.2938 - val_acc: 0.1184
Epoch 2/4
44848/44848 [==============================] - 21s - loss: 1.6131 - acc: 0.4180 - val_loss: 2.2959 - val_acc: 0.1151
Epoch 3/4
44848/44848 [==============================] - 21s - loss: 1.5957 - acc: 0.4224 - val_loss: 2.2986 - val_acc: 0.1140
Epoch 4/4
44848/44848 [==============================] - 21s - loss: 1.5758 - acc: 0.4292 - val_loss: 2.2982 - val_acc: 0.1140


In [30]:
bn_model.optimizer.lr=0.0001

In [32]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=10, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/10
44848/44848 [==============================] - 21s - loss: 1.5107 - acc: 0.4486 - val_loss: 2.2990 - val_acc: 0.1135
Epoch 2/10
44848/44848 [==============================] - 21s - loss: 1.5021 - acc: 0.4540 - val_loss: 2.2980 - val_acc: 0.1062
Epoch 3/10
44848/44848 [==============================] - 21s - loss: 1.4902 - acc: 0.4565 - val_loss: 2.2989 - val_acc: 0.1028
Epoch 4/10
44848/44848 [==============================] - 21s - loss: 1.4814 - acc: 0.4577 - val_loss: 2.2981 - val_acc: 0.1137
Epoch 5/10
44848/44848 [==============================] - 20s - loss: 1.4741 - acc: 0.4598 - val_loss: 2.2985 - val_acc: 0.1137
Epoch 6/10
44848/44848 [==============================] - 20s - loss: 1.4722 - acc: 0.4603 - val_loss: 2.2993 - val_acc: 0.1135
Epoch 7/10
44848/44848 [==============================] - 20s - loss: 1.4621 - acc: 0.4631 - val_loss: 2.2998 - val_acc: 0.1019
Epoch 8/10
44848/44848 [==============================]

In [33]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

## Pseudo labeling
Try using a combination of pseudo labeling and knowledge distillation to allow us to use unlabeled data (semi-supervised learning).    For this experiment, we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set.

AT a later date we'll try using the test set.


To do this, we calculate the predictions of our model...

In [34]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

concatenate with the training models

In [35]:
comb_psuedo = np.concatenate([da_trn_labels, val_pseudo])

In [36]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

In [37]:
save_array(path+'results/com_feat.dat', comb_feat)

In [38]:
save_array(path+'results/comb_psuedo.dat', comb_psuedo)

In [13]:
comb_feat = load_array(path+'results/com_feat.dat')

fine-tune the model using this data

In [12]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [39]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/1
49332/49332 [==============================] - 22s - loss: 1.5266 - acc: 0.4467 - val_loss: 2.2992 - val_acc: 0.1135


In [40]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/4
49332/49332 [==============================] - 22s - loss: 1.5202 - acc: 0.4456 - val_loss: 2.2992 - val_acc: 0.1135
Epoch 2/4
49332/49332 [==============================] - 22s - loss: 1.5218 - acc: 0.4438 - val_loss: 2.2985 - val_acc: 0.1135
Epoch 3/4
49332/49332 [==============================] - 23s - loss: 1.5160 - acc: 0.4489 - val_loss: 2.2990 - val_acc: 0.0999
Epoch 4/4
49332/49332 [==============================] - 22s - loss: 1.5158 - acc: 0.4449 - val_loss: 2.2986 - val_acc: 0.1135


In [41]:
bn_model.optimizer.lr=0.00001

In [42]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/4
49332/49332 [==============================] - 23s - loss: 1.5114 - acc: 0.4490 - val_loss: 2.2990 - val_acc: 0.1019
Epoch 2/4
49332/49332 [==============================] - 22s - loss: 1.5084 - acc: 0.4480 - val_loss: 2.2984 - val_acc: 0.1135
Epoch 3/4
49332/49332 [==============================] - 23s - loss: 1.5014 - acc: 0.4497 - val_loss: 2.2984 - val_acc: 0.1135
Epoch 4/4
49332/49332 [==============================] - 23s - loss: 1.5025 - acc: 0.4518 - val_loss: 2.3000 - val_acc: 0.1028


In [43]:
bn_model.save_weights(path+'models/bn_ps8.h5')

## Submit
clip the amount using the validation set, prior to submitting

In [14]:
bn_model.load_weights(path+'models/bn_ps8.h5')

In [21]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [22]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

NameError: name 'val_preds' is not defined

In [17]:
conv_test_feat=load_array(path+'results/conv_test_feat.dat')

In [18]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [29]:
subm = do_clip(preds,0.93)

In [24]:
subm_name = path + 'results/subm.gz'

In [27]:
classes = sorted(batches.class_indices, key = batches.class_indices.get)

In [36]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.110313,0.097777,0.109244,0.098382,0.112967,0.103934,0.106074,0.084788,0.085437,0.091085
1,img_14887.jpg,0.186423,0.103934,0.036819,0.043997,0.042982,0.091466,0.050582,0.061697,0.125653,0.256448
2,img_62885.jpg,0.110313,0.097777,0.109244,0.098382,0.112967,0.103934,0.106074,0.084788,0.085437,0.091085
3,img_45125.jpg,0.105365,0.095788,0.101636,0.093413,0.121660,0.097205,0.113401,0.089760,0.090306,0.091467
4,img_22633.jpg,0.110313,0.097777,0.109244,0.098382,0.112967,0.103934,0.106074,0.084788,0.085437,0.091085


In [37]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [38]:
FileLink(subm_name)

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/results/subm.gz